# Speech Recognition Training Notebook




This notebook demonstrates how to train a 20kb [Simple Audio Recognition](https://www.tensorflow.org/tutorials/sequences/audio_recognition) model for [TensorFlow Lite for Microcontrollers](https://tensorflow.org/lite/microcontrollers/overview). It will produce the same model used in the [micro_speech](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/experimental/micro/examples/micro_speech) example application.

The model is designed to be used with [Google Colaboratory](https://colab.research.google.com).

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1cR5JJNYzWLlgSK9c16nnFLhxtW_bmO-p#offline=true&sandboxMode=true"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/experimental/micro/examples/micro_speech/train_speech_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>




The notebook runs Python scripts to train and freeze the model, and uses the TensorFlow Lite converter to convert it for use with TensorFlow Lite for Microcontrollers.

**Training is much faster using GPU acceleration.** Before you proceed, ensure you are using a GPU runtime by going to **Runtime -> Change runtime type** and selecting **GPU**. Training 18,000 iterations will take 1.5-2 hours on a GPU runtime.

## Configure training

The following `os.environ` lines can be customized to set the words that will be trained for, and the steps and learning rate of the training. The default values will result in the same model that is used in the micro_speech example. Run the cell to set the configuration:

In [ ]:
import os

# A comma-delimited list of the words you want to train for.
# The options are: yes,no,up,down,left,right,on,off,stop,go
# All other words will be used to train an "unknown" category.
WANTED_WORDS = ["yes", "no"]

# The number of steps and learning rates can be specified as comma-separated
# lists to define the rate at each stage. For example,
# TRAINING_STEPS=15000,3000 and LEARNING_RATE=0.001,0.0001
# will run 18,000 training loops in total, with a rate of 0.001 for the first
# 15,000, and 0.0001 for the final 3,000.
TRAINING_STEPS = [15000, 3000]
LEARNING_RATE = ["0.001", "0.0001"]

# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
TOTAL_STEPS = sum(TRAINING_STEPS)

# Print the configuration to confirm it
!echo "Training these words: {WANTED_WORDS}"
!echo "Training steps in each stage: {TRAINING_STEPS}"
!echo "Learning rate in each stage: {LEARNING_RATE}"
!echo "Total number of training steps: {TOTAL_STEPS}"

Training these words: ['yes', 'no']
Training steps in each stage: [15000, 3000]
Learning rate in each stage: ['0.001', '0.0001']
Total number of training steps: 18000


# Connect to Google Drive

This section connects the notebook with your Google Drive. Since Colab instances are ephemeral, we need somewhere more permanent to store the trained ML model. Running this cell mounts your Google Drive just like a regular folder at `/content/drive`.

In [ ]:
import os.path
from google.colab import drive

DRIVE_STORAGE_PATH = '/content/drive/My Drive/speech-recognition'

def ensure_drive():
  if not os.path.exists('/content/drive/My Drive'):
    drive.mount('/content/drive', force_remount=True)
    
ensure_drive()

## Install dependencies

Next, we'll install a GPU build of TensorFlow, so we can use GPU acceleration for training.

In [ ]:
# Replace Colab's default TensorFlow install with the pinned version used
# for speech training.
!pip uninstall -y tensorflow tensorflow_estimatorpip uninstall -y tensorflow tensorflow_estimator tensorboard
!pip install -q tf-estimator-nightly==1.14.0.dev2019072901 tf-nightly-gpu==1.15.0.dev20190729

Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0
Uninstalling tensorflow-estimator-1.15.1:
  Successfully uninstalled tensorflow-estimator-1.15.1
Uninstalling tensorboard-1.15.0:
  Successfully uninstalled tensorboard-1.15.0
     |████████████████████████████████| 501kB 2.8MB/s 
     |████████████████████████████████| 406.6MB 41kB/s 
     |████████████████████████████████| 3.8MB 43.3MB/s 


## Download TensorFlow

We'll also clone the TensorFlow repository, which contains the scripts that train and freeze the model.

In [ ]:
# Clone the repository from GitHub
!git clone -q https://github.com/tensorflow/tensorflow
# Check out a commit that has been tested to work
# with the build of TensorFlow we're using
!git -c advice.detachedHead=false -C tensorflow checkout -f 17ce384df70

import subprocess
subprocess.run(
    ['git', '-C', 'tensorflow', 'apply', '-'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    encoding='utf-8',
    check=True,
    input="""\
diff --git a/tensorflow/examples/speech_commands/train.py b/tensorflow/examples/speech_commands/train.py
index 446e351cb8..431c2d3d34 100644
--- a/tensorflow/examples/speech_commands/train.py
+++ b/tensorflow/examples/speech_commands/train.py
@@ -234,7 +234,7 @@ def main(_):
             dropout_prob: 0.5
         })
     train_writer.add_summary(train_summary, training_step)
-    tf.compat.v1.logging.info(
+    tf.compat.v1.logging.debug(
         'Step #%d: rate %f, accuracy %.1f%%, cross entropy %f' %
         (training_step, learning_rate_value, train_accuracy * 100,
          cross_entropy_value))
""")

Checking out files: 100% (9047/9047), done.
HEAD is now at 17ce384df7 Share ownership of `UnboundedWorkQueue` between collective executor and executor manager.


CompletedProcess(args=['git', '-C', 'tensorflow', 'apply', '-'], returncode=0, stdout='', stderr='')

### Optional: Visualize graph and training rate

In [ ]:
!mkdir -p /content/retrain_logs
%load_ext tensorboard
%tensorboard --logdir /content/retrain_logs

## Create trained model

In [ ]:
import datetime

wanted_words_str = ','.join(WANTED_WORDS)
training_steps_str = ','.join([str(x) for x in TRAINING_STEPS])
learning_rate_str = ','.join(LEARNING_RATE)

TRAINING_RUN_NAME="{}-{}".format(
    wanted_words_str, datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))
TRAIN_DIR = f'{DRIVE_STORAGE_PATH}/{TRAINING_RUN_NAME}'
!mkdir -p "{TRAIN_DIR}"

# Suppress INFO messages from C++ logging--these will slow down Colab and 
# eventually crash the browser.
#--verbosity=INFO \
!python tensorflow/tensorflow/examples/speech_commands/train.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words={wanted_words_str} --silence_percentage=25 \
--unknown_percentage=25 --quantize=1 \
--how_many_training_steps={training_steps_str} \
--learning_rate={learning_rate_str} --summaries_dir=/content/retrain_logs \
--data_dir=/content/speech_dataset --train_dir="{TRAIN_DIR}"

!python tensorflow/tensorflow/examples/speech_commands/freeze.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words={wanted_words_str} --quantize=1 \
--output_file="{TRAIN_DIR}/tiny_conv.pb" \
--start_checkpoint="{TRAIN_DIR}/tiny_conv.ckpt-{TOTAL_STEPS}"

!toco \
--graph_def_file="{TRAIN_DIR}/tiny_conv.pb" \
--output_file="{TRAIN_DIR}/tiny_conv.tflite" \
--input_shapes=1,49,40,1 --input_arrays=Reshape_2 \
--output_arrays='labels_softmax' \
--inference_type=QUANTIZED_UINT8 --mean_values=0 --std_dev_values=9.8077

print('Training completed')
print(f'The frozen graph is: {TRAIN_DIR}/tiny_conv.pb')
print(f'The tflite graph is: {TRAIN_DIR}/tiny_conv.tflite')

2019-11-07 18:07:26.441261: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-07 18:07:26.468283: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-07 18:07:26.633034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-07 18:07:26.634206: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2a32a00 executing computations on platform CUDA. Devices:
2019-11-07 18:07:26.634239: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2019-11-07 18:07:26.637025: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-11-07 18:07:26.637402: I tensorflow/compil

## Spot checks

In [ ]:
import os.path
if not os.path.exists('/content/speech_dataset'):
  !wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
  !mkdir /content/speech_dataset
  !tar -xzf speech_commands_v0.02.tar.gz -C /content/speech_dataset

ensure_drive()


for phrase in ('yes', 'no', 'right'):
  wav = !ls -1 /content/speech_dataset/{phrase}/*.wav | head -n 1
  wav = wav[0]

  print('')
  print(f'{phrase:10s} <' + '-' * 69)
  !cd tensorflow && python tensorflow/examples/speech_commands/label_wav.py --graph="{TRAIN_DIR}/tiny_conv.pb" --labels="{TRAIN_DIR}/tiny_conv_labels.txt" --wav={wav} 2>/dev/null



yes        <---------------------------------------------------------------------
yes (score = 0.99978)
_unknown_ (score = 0.00020)
no (score = 0.00003)

no         <---------------------------------------------------------------------
no (score = 0.96930)
_unknown_ (score = 0.02235)
yes (score = 0.00833)

right      <---------------------------------------------------------------------
_unknown_ (score = 0.99154)
no (score = 0.00779)
yes (score = 0.00063)
